# 유사 문서 검색 실습 (검색 품질 심화)

* Top-K 검색 결과의 의미와 성능 차이를 이해하고,
* 검색 유사도 점수(score) 개념을 학습하며,
* 검색 품질을 조정하기 위한 실습을 진행합니다.

### 1. 사전 준비

In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
import numpy as np

load_dotenv()
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

# 저장된 인덱스 불러오기
faiss_index = FAISS.load_local(
    "embeddings/faiss_index_e5",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)
retriever = faiss_index.as_retriever()

### 2. Top-K 검색 결과 비교 실습
- **Tip**: K가 높아질수록 더 많은 문서를 참조하지만, 모델이 혼란스러워질 수도 있습니다. (답변 품질 저하)

In [13]:
query = "국회의 임기는 몇 년인가요?"

# 다양한 K 값 실험
for k in [1, 3, 5]:
    print(f"\n--- Top-{k} 검색 결과 ---")
    retriever.search_kwargs["k"] = k
    results = retriever.get_relevant_documents(query)
    
    for i, doc in enumerate(results, 1):
        print(f"{i}.", doc.page_content.strip()[:100], "...")


--- Top-1 검색 결과 ---
1. ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다. ...

--- Top-3 검색 결과 ---
1. ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다. ...
2. 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최 ...
3. ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다. ...

--- Top-5 검색 결과 ---
1. ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다. ...
2. 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최 ...
3. ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다. ...
4. 제86조 ①국무총리는 국회의 동의를 얻어 대통령이 임명한다. ...
5. 제104조 ①대법원장은 국회의 동의를 얻어 대통령이 임명한다. ...


### 3. 유사도 점수 확인

LangChain 기본 retriever에서는 점수를 직접 제공하지 않지만,
**FAISS 객체 내부에서 직접 벡터 간 거리 계산으로 유사도 점수**를 확인할 수 있습니다.

- 점수가 낮을수록 유사도가 높은 것입니다 (L2 거리 기준)
- FAISS는 기본적으로 거리 기반 검색을 수행합니다.

In [14]:
import numpy as np

# 쿼리 임베딩 구하기
query = "국회의 임기는 몇 년인가요?"
query_vec = embedding.embed_query(query)

# Top-K 문서 가져오기
k = 3
retriever.search_kwargs["k"] = k
results = retriever.get_relevant_documents(query)

# 문서의 임베딩 벡터도 직접 계산
doc_vectors = [embedding.embed_query(doc.page_content) for doc in results]

# L2 거리 계산 및 출력
print(f"\n🔍 Top-{k} 검색 결과 (L2 거리 기준 유사도):\n")
for i, (doc, vec) in enumerate(zip(results, doc_vectors), 1):
    distance = np.linalg.norm(np.array(query_vec) - np.array(vec))
    print(f"{i}. 거리: {distance:.4f}")
    print("   문장:", doc.page_content.strip()[:100], "...\n")


🔍 Top-3 검색 결과 (L2 거리 기준 유사도):

1. 거리: 0.4656
   문장: ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다. ...

2. 거리: 0.4820
   문장: 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최 ...

3. 거리: 0.4896
   문장: ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다. ...



In [15]:
import numpy as np

query = "국회의 임기는 몇 년인가요?"
query_vec = embedding.embed_query(query)

# Top-K 설정
k = 5
retriever.search_kwargs["k"] = k
results = retriever.get_relevant_documents(query)

# 거리 계산 및 필터링
threshold = 1.0  # 이 값보다 가까운(=유사한) 문서만 필터링
filtered = []

for doc in results:
    doc_vec = embedding.embed_query(doc.page_content)
    distance = np.linalg.norm(np.array(query_vec) - np.array(doc_vec))
    
    if distance <= threshold:
        filtered.append((distance, doc.page_content.strip()))

# 출력
print(f"\n🎯 L2 거리 ≤ {threshold} 인 문서만 출력 (Top-{k} 중에서 필터링)\n")
for i, (distance, content) in enumerate(filtered, 1):
    print(f"{i}. 거리: {distance:.4f}")
    print("   문장:", content[:100], "...\n")


🎯 L2 거리 ≤ 1.0 인 문서만 출력 (Top-5 중에서 필터링)

1. 거리: 0.4656
   문장: ②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임할 수 있다. ...

2. 거리: 0.4820
   문장: 제3조 ①이 헌법에 의한 최초의 국회의원선거는 이 헌법공포일로부터 6월 이내에 실시하며, 이 헌법에 의하여 선출된 최초의 국회의원의 임기는 국회의원선거후 이 헌법에 의한 국회의 최 ...

3. 거리: 0.4896
   문장: ②이 헌법공포 당시의 국회의원의 임기는 제1항에 의한 국회의 최초의 집회일 전일까지로 한다. ...

4. 거리: 0.4897
   문장: 제86조 ①국무총리는 국회의 동의를 얻어 대통령이 임명한다. ...

5. 거리: 0.4986
   문장: 제104조 ①대법원장은 국회의 동의를 얻어 대통령이 임명한다. ...



### 4. 정리 및 다음 단계

* Top-K 값에 따라 결과 품질이 달라질 수 있습니다.
* 유사도 점수를 기준으로 검색 결과의 신뢰도를 판단할 수 있습니다.
* 검색 품질을 높이려면 다음도 고려해야 합니다:

  * 적절한 Chunking
  * 임베딩 모델 선택
  * 문서 전처리